# **Análise de Coerência do Projeto até o Momento**

Até esta etapa, o projeto foi desenvolvido em total alinhamento com as diretrizes propostas. Após a definição do objetivo e das perguntas de negócio, foi selecionado um conjunto de dados adequado, coletado e armazenado corretamente na plataforma Databricks. A ingestão foi ajustada para lidar com estruturas complexas de CSV e os dados foram organizados conforme a arquitetura Bronze, Prata e Ouro.

A modelagem foi construída no formato Snowflake, contemplando dimensões, fatos e tabelas de relacionamento, garantindo consistência e suporte às análises planejadas. Todas as transformações necessárias foram documentadas e executadas de forma ordenada, assegurando a preparação dos dados para a próxima fase.

Com isso, o projeto está tecnicamente consistente para avançar para a etapa de análise dos dados, onde serão verificadas a qualidade e a integridade das informações e, em seguida, respondidas as perguntas definidas no início do trabalho.

In [0]:
CREATE OR REPLACE TABLE mvp.gold.qualidade_nulos AS
SELECT 
COUNT(*) AS total_registros,

SUM(CASE WHEN id_titulo IS NULL THEN 1 ELSE 0 END) AS nulos_id,
SUM(CASE WHEN titulo IS NULL THEN 1 ELSE 0 END) AS nulos_titulo,
SUM(CASE WHEN show_type IS NULL THEN 1 ELSE 0 END) AS nulos_show_type,
SUM(CASE WHEN ano_lancamento IS NULL THEN 1 ELSE 0 END) AS nulos_ano_lancamento,
SUM(CASE WHEN duracao IS NULL THEN 1 ELSE 0 END) AS nulos_duracao,
SUM(CASE WHEN temporadas IS NULL THEN 1 ELSE 0 END) AS nulos_temporadas,
SUM(CASE WHEN imdb_score IS NULL THEN 1 ELSE 0 END) AS nulos_imdb_score,
SUM(CASE WHEN imdb_votes IS NULL THEN 1 ELSE 0 END) AS nulos_imdb_votes,
SUM(CASE WHEN tmdb_score IS NULL THEN 1 ELSE 0 END) AS nulos_tmdb_score,
SUM(CASE WHEN tmdb_popularity IS NULL THEN 1 ELSE 0 END) AS nulos_tmdb_popularity
FROM dim_titulo;




In [0]:
SELECT * FROM mvp.gold.qualidade_nulos

In [0]:
CREATE OR REPLACE TABLE mvp.gold.qualidade_vazios AS
SELECT
SUM(CASE WHEN title = '' THEN 1 END) AS vazios_title,
SUM(CASE WHEN category = '' THEN 1 END) AS vazios_category,
SUM(CASE WHEN genres = '[]' THEN 1 END) AS vazios_genres,
SUM(CASE WHEN production_countries = '[]' THEN 1 END) AS vazios_production_countries
FROM mvp.silver.titles;


In [0]:
SELECT * FROM mvp.gold.qualidade_vazios

In [0]:
CREATE OR REPLACE TABLE mvp.gold.qualidade_duplicidade AS
SELECT 
id,
COUNT(*) AS repeticoes
FROM mvp.silver.titles
GROUP BY id
HAVING COUNT(*) > 1;


In [0]:
SELECT * FROM mvp.gold.qualidade_duplicidade

In [0]:
CREATE OR REPLACE TABLE mvp.gold.qualidade_estatisticas AS
SELECT
MIN(ano_lancamento) AS min_ano_lancamento,
MAX(ano_lancamento) AS max_ano_lancamento,
MIN(duracao) AS min_duracao,
MAX(duracao) AS max_duracao,
MIN(temporadas) AS min_temporada,
MAX(temporadas) AS max_temporada,
MIN(imdb_score) AS min_imdb_score,
MAX(imdb_score) AS max_imdb_score,
MIN(imdb_votes) AS min_imdb_votes,
MAX(imdb_votes) AS max_imdb_votes,
MIN(tmdb_score) AS min_tmdb_score,
MAX(tmdb_score) AS max_tmdb_score,
MIN(tmdb_popularity) AS min_tmdb_popularity,
MAX(tmdb_popularity) AS max_tmdb_popularity
FROM dim_titulo;


In [0]:
SELECT * FROM mvp.gold.qualidade_estatisticas

In [0]:
CREATE OR REPLACE TABLE mvp.gold.qualidade_tipos_invalidos AS
SELECT
    -- anos inválidos: antes de 1800 ou depois de 1 ano à frente do ano atual
    SUM(CASE 
            WHEN release_year < 1800 
              OR release_year > YEAR(current_date()) + 1 
            THEN 1 
        END) AS anos_invalidos,

    -- durações inválidas
    SUM(CASE 
            WHEN runtime < 0 
            THEN 1 
        END) AS duracoes_invalidas,

    -- temporadas inválidas
    SUM(CASE 
            WHEN seasons < 0 
            THEN 1 
        END) AS temporadas_invalidas,

    -- notas IMDb inválidas (<0 ou >10)
    SUM(CASE 
            WHEN imdb_score < 0 
              OR imdb_score > 10 
            THEN 1 
        END) AS imdb_invalidos,

    -- notas TMDb inválidas (<0 ou >10)
    SUM(CASE 
            WHEN tmdb_score < 0 
              OR tmdb_score > 10 
            THEN 1 
        END) AS tmdb_invalidos

FROM mvp.silver.titles;



In [0]:
SELECT * FROM mvp.silver.qualidade_tipos_invalidos

## Resumo da Análise de Qualidade dos Dados

A análise de qualidade foi conduzida sobre o conjunto de dados já padronizado na camada Silver, avaliando cinco dimensões principais: valores nulos, valores vazios, duplicidades, estatísticas descritivas e tipos inválidos.

**Valores Nulos**

A tabela qualidade_nulos mostrou:

O dataset contém 3030 registros.

Colunas essenciais como id, title, category, release_year, runtime, genres e production_countries não possuem valores nulos, garantindo consistência das informações principais.

Há presença de nulos em:

seasons → 2408 valores nulos (esperado, pois filmes não possuem temporadas)

imdb_score → alguns nulos

imdb_votes → alguns nulos

tmdb_score → alguns nulos

tmdb_popularity → alguns nulos

Os valores nulos nas colunas tmdb_score, tmdb_popularity, imdb_score e imdb_votes representam títulos cujas pontuações não puderam ser coletadas nas respectivas fontes (TMDb e IMDb), seja por ausência de avaliação, inexistência do título na base ou indisponibilidade da informação no momento da ingestão.”


**Valores Vazios**

A tabela qualidade_vazios identificou:

genres → 35 listas vazias

production_countries → 56 listas vazias
Esses casos indicam apenas que a fonte não forneceu informações, não representando erro estrutural.


**Duplicidades**

A tabela qualidade_duplicidade não encontrou registros duplicados, confirmando que a chave id é única e que não há redundância no dataset.


**Estatísticas Descritivas**

A tabela qualidade_estatisticas apresentou:

release_year: varia entre 1901 e 2023

runtime: entre 1 e 299 minutos

seasons: entre 1 e 53 temporadas

imdb_score: entre 1.5 e 9.4

imbd_votes: entre 5 e 2.684.317

tmdb_score: entre 0 e 10

tmdb_popularity: entre 0.050 e 3481.253

Todos os valores estão dentro de intervalos plausíveis, sem anomalias numéricas.


**Tipos Inválidos**

A tabela qualidade_tipos_invalidos retornou zero ocorrências, indicando que:

Não há anos inválidos

Não há durações incorretas

Não há temporadas inconsistentes

Scores IMDb e TMDB são válidos


**Conclusão Geral**

A análise confirma que o conjunto de dados possui boa qualidade, sem inconsistências estruturais ou valores que comprometam a análise.

Algumas colunas naturalmente apresentam nulos ou listas vazias devido ao contexto da obra (ex.: filmes não têm temporadas) ou ausência de dados nas fontes originais, mas isso não interfere na modelagem nem nas análises futuras.

Com esses resultados, o dataset está pronto para a etapa final de análise exploratória, onde as perguntas definidas no MVP serão respondidas.

In [0]:
USE CATALOG mvp;
USE SCHEMA gold

**Títulos mais populares populares segundo IMDb**

In [0]:
SELECT 
titulo,
show_type,
imdb_score,
imdb_votes,
ano_lancamento
FROM dim_titulo
ORDER BY imdb_score DESC, imdb_votes DESC
LIMIT 10;



A análise de popularidade identificou os títulos que combinam as maiores avaliações no IMDb com elevado volume de votos, refletindo forte aceitação crítica e ampla participação do público. Produções como Chernobyl, Band of Brothers, The Shawshank Redemption, The Wire, Game of Thrones e The Sopranos destacam-se de forma consistente no topo do ranking. Além das notas elevadas, esses títulos possuem centenas de milhares ou até milhões de votos, reforçando sua relevância cultural e forte impacto entre os usuários. O padrão observado indica que os títulos mais populares tendem a unir qualidade cinematográfica, alto alcance e longevidade de interesse ao longo dos anos.

**Títulos menos populares segundo IMDb**

In [0]:
SELECT 
titulo,
show_type,
imdb_score,
imdb_votes,
ano_lancamento
FROM dim_titulo
WHERE imdb_score IS NOT NULL
AND imdb_votes IS NOT NULL
ORDER BY imdb_score ASC, imdb_votes ASC
LIMIT 10;


Os resultados apresentam os 10 títulos menos populares segundo o IMDb, caracterizados por baixas pontuações (entre 1,5 e 3,4) e, em vários casos, baixo volume de votos, o que indica rejeição do público e/ou baixa representatividade das avaliações. Observa-se a presença de filmes e séries, majoritariamente lançados a partir de 2008, sugerindo que a baixa popularidade não está associada a produções muito antigas, mas sim à percepção negativa do público ou à baixa adesão.

**Títulos mais populares segundo TMDb**

In [0]:
SELECT
titulo,
show_type,
tmdb_score,
tmdb_popularity,
ano_lancamento
FROM dim_titulo
WHERE tmdb_score IS NOT NULL
ORDER BY
tmdb_score DESC,
tmdb_popularity DESC
LIMIT 10;



Os dados apresentam os títulos mais bem avaliados no TMDb, todos com pontuação máxima (tmdb_score = 10). Como critério de desempate, a popularidade (tmdb_popularity) permitiu ordenar os títulos por relevância, destacando aqueles com maior visibilidade e engajamento do público. Observa-se a presença de filmes e séries, indicando que a avaliação máxima não se restringe a um único formato, e reforça que a combinação entre nota e popularidade produz um ranking mais representativo do que o uso isolado da pontuação.


**Títulos menos populares TMDb**

In [0]:
SELECT  
titulo,
show_type,
tmdb_score,
tmdb_popularity,
ano_lancamento
FROM dim_titulo
WHERE tmdb_score IS NOT NULL
ORDER BY
tmdb_score ASC,
tmdb_popularity ASC
LIMIT 10;


Os dados apresentam os títulos menos populares segundo o TMDb, caracterizados por pontuações muito baixas (entre 1 e 2) e baixa popularidade, indicando pouca aceitação e reduzido engajamento do público. Observa-se a presença tanto de filmes quanto de séries, sem concentração em um único formato, o que reforça que a baixa avaliação não está restrita a um tipo específico de conteúdo, mas sim à percepção negativa do público.

**Filmes mais populares segundo IMDB**

In [0]:
SELECT
titulo,
imdb_score,
imdb_votes,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'MOVIE'
AND imdb_score IS NOT NULL
ORDER BY imdb_score DESC, imdb_votes DESC
LIMIT 10;




Os dados apresentam os filmes mais bem avaliados segundo o IMDb, com pontuações elevadas (entre 8,8 e 9,3). Observa-se a predominância de títulos amplamente consagrados, com alto volume de votos, o que reforça a representatividade das avaliações. Também aparecem alguns títulos com baixo número de votos ou valores nulos, indicando que, apesar da nota alta, essas avaliações podem ter menor peso estatístico. Em geral, o ranking evidencia produções reconhecidas pelo público e pela crítica, principalmente lançadas entre as décadas de 1990 e 2010.


**Filmes menos populares segundo o IMDB**

In [0]:
SELECT
titulo,
imdb_score,
imdb_votes,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'MOVIE'
AND imdb_score IS NOT NULL
ORDER BY
imdb_score ASC,
imdb_votes ASC
LIMIT 10;


Os dados apresentam os filmes menos bem avaliados segundo o IMDb, com pontuações baixas variando entre 2,5 e 3,7. Observa-se que, mesmo com volumes relevantes de votos em alguns casos, as avaliações permanecem negativas, indicando rejeição consistente do público. Os títulos abrangem diferentes períodos de lançamento, o que sugere que a baixa avaliação não está associada a uma época específica, mas sim à percepção desfavorável dos espectadores.


**Séries mais populares segundo o IMDB**


In [0]:
SELECT
titulo,
imdb_score,
imdb_votes,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'SHOW'
AND imdb_score IS NOT NULL
ORDER BY
imdb_score DESC,
imdb_votes DESC
LIMIT 10;


Os dados apresentam as séries mais bem avaliadas segundo o IMDb, com pontuações muito elevadas (entre 9,0 e 9,4). Destacam-se produções amplamente reconhecidas, com alto volume de votos, o que reforça a consistência e representatividade das avaliações. Também aparecem títulos com menor quantidade de votos, indicando avaliações positivas, porém com menor peso estatístico. No conjunto, o ranking evidencia séries de forte impacto cultural e reconhecimento do público ao longo de diferentes períodos de lançamento.

**Séries menos populares segundo o IMDB**


In [0]:
SELECT
titulo,
imdb_score,
imdb_votes,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'SHOW'
AND imdb_score IS NOT NULL
ORDER BY
imdb_score ASC,
imdb_votes ASC
LIMIT 10;


Os dados apresentam as séries menos bem avaliadas segundo o IMDb, com pontuações baixas variando entre 1,5 e 4,4. Observa-se que algumas produções possuem volume expressivo de votos, o que indica que a baixa avaliação reflete uma percepção negativa consistente do público, e não apenas avaliações pontuais. Os títulos são majoritariamente lançamentos recentes, sugerindo que a rejeição não está associada à antiguidade das obras, mas sim à qualidade percebida ou à recepção crítica desfavorável.


**Filmes mais populares segundo o TMDB**

In [0]:
SELECT
titulo,
tmdb_score,
tmdb_popularity,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'MOVIE'
AND tmdb_score IS NOT NULL
ORDER BY
tmdb_score DESC,
tmdb_popularity DESC
LIMIT 10;


Os dados apresentam os filmes mais bem avaliados segundo o TMDb, todos com pontuação máxima (tmdb_score = 10). Diante do empate nas notas, a métrica de popularidade (tmdb_popularity) foi utilizada como critério de desempate, evidenciando diferenças de alcance e engajamento do público entre os títulos. Observa-se a presença de produções de diferentes períodos de lançamento, indicando que a avaliação máxima não está restrita a uma época específica, mas a combinação entre nota e popularidade fornece um ranking mais representativo dos conteúdos.

**Filmes menos populares segundo o TMDB**

In [0]:
SELECT
titulo,
tmdb_score,
tmdb_popularity,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'MOVIE'
AND tmdb_score IS NOT NULL
ORDER BY
tmdb_score ASC,
tmdb_popularity ASC
LIMIT 10;


Os dados apresentam os filmes menos bem avaliados segundo o TMDb, com pontuações muito baixas (entre 1 e 2) e níveis reduzidos de popularidade, indicando baixa aceitação e pouco engajamento do público. Observa-se a predominância de títulos lançados recentemente, o que sugere que a baixa avaliação não está relacionada à antiguidade das produções, mas sim à percepção negativa do público em relação à qualidade do conteúdo.

**Séries mais populares segundo o TMDB**

In [0]:
SELECT
titulo,
tmdb_score,
tmdb_popularity,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'SHOW'
AND tmdb_score IS NOT NULL
ORDER BY
tmdb_score DESC,
tmdb_popularity DESC
LIMIT 10;


Os dados apresentam as séries mais bem avaliadas segundo o TMDb, com predominância de pontuação máxima (tmdb_score = 10). Como critério de desempate, a popularidade (tmdb_popularity) evidencia diferenças relevantes de alcance e engajamento, destacando títulos mais conhecidos frente a outros com menor visibilidade, apesar da mesma nota. Observa-se ainda uma variação significativa no ano de lançamento, indicando que a alta avaliação no TMDb não está associada a uma geração específica de séries, mas sim à percepção positiva do público combinada à relevância do conteúdo.

**Séries menos populares segundo o TMDB**

In [0]:
SELECT
titulo,
tmdb_score,
tmdb_popularity,
ano_lancamento
FROM dim_titulo
WHERE show_type = 'SHOW'
AND tmdb_score IS NOT NULL
ORDER BY tmdb_score ASC, tmdb_popularity ASC
LIMIT 10;


Os dados apresentam as séries menos bem avaliadas segundo o TMDb, com pontuações muito baixas (entre 1 e 2). Observa-se que, apesar das notas reduzidas, alguns títulos possuem níveis relativamente elevados de popularidade, indicando alto alcance, mas recepção negativa do público. A maioria das produções é recente, concentrada nos anos de 2020 a 2022, o que sugere que a baixa avaliação está mais associada à qualidade percebida do conteúdo do que à falta de visibilidade ou ao tempo de lançamento.

**Comparativo geral dos resultados**

De forma geral, observa-se convergência parcial entre IMDb e TMDb quanto aos títulos mais e menos populares, mas com diferenças relevantes na forma como a popularidade se manifesta em cada plataforma.

**Filmes**

Mais populares:
No IMDb, os filmes mais bem avaliados tendem a ser clássicos amplamente consagrados, com altíssimos volumes de votos, o que confere forte representatividade às avaliações.
No TMDb, embora muitos títulos também apresentem pontuação máxima, a popularidade varia significativamente, indicando que nem todos os filmes com nota máxima possuem o mesmo nível de alcance ou engajamento.

Menos populares:
Em ambas as plataformas, os filmes menos populares apresentam pontuações baixas de forma consistente, inclusive em títulos com quantidade relevante de votos, o que sugere rejeição consolidada do público, e não apenas avaliações pontuais.

**Séries**

Mais populares:
No IMDb, as séries mais bem avaliadas combinam notas elevadas e grande volume de votos, refletindo forte reconhecimento do público e impacto cultural duradouro.
No TMDb, há maior concentração de pontuações máximas, sendo a popularidade o principal fator de diferenciação, revelando diferenças de visibilidade entre séries igualmente bem avaliadas.

Menos populares:
Tanto no IMDb quanto no TMDb, as séries menos populares são majoritariamente lançamentos recentes, com notas baixas, indicando que a rejeição não está associada à antiguidade, mas à recepção negativa do público. No TMDb, destaca-se ainda a presença de séries com popularidade relativamente alta, mas avaliações ruins, evidenciando alto alcance sem aprovação crítica.

**Conclusão comparativa**

Em síntese, o IMDb se destaca por refletir avaliações mais consolidadas, sustentadas por grande volume de votos, enquanto o TMDb evidencia melhor o alcance e a visibilidade dos títulos, por meio da métrica de popularidade. A combinação entre pontuação e métricas de representatividade mostrou-se essencial para uma análise mais equilibrada, evitando interpretações distorcidas baseadas apenas na nota.

**Títulos produzidos no Brasil**


In [0]:
SELECT 
titulo,
show_type,
tmdb_score,
ano_lancamento
FROM 
tabela_pais tp
JOIN 
mvp.gold.dim_titulo dt
ON tp.id_titulo = dt.id_titulo
JOIN 
Dim_pais dp 
ON dp.id_pais = tp.id_pais
WHERE 
dp.pais = 'BR';


Os resultados identificam corretamente os títulos produzidos no Brasil, a partir das tabelas da camada ouro, utilizando a dimensão de países e a tabela de relacionamento título–país. A consulta confirma a consistência do modelo dimensional e permite analisar as produções brasileiras de forma integrada às métricas de avaliação e popularidade.

**Títulos brasileiros mais populares**

In [0]:
SELECT
dt.titulo,
dt.show_type,
dt.imdb_score,
dt.imdb_votes,
dt.ano_lancamento
FROM tabela_pais tp
JOIN dim_pais dp
ON tp.id_pais = dp.id_pais
JOIN dim_titulo dt
ON tp.id_titulo = dt.id_titulo
WHERE dp.pais = 'BR'
AND dt.imdb_score IS NOT NULL
ORDER BY
dt.imdb_score DESC,
dt.imdb_votes DESC
LIMIT 10;


In [0]:
SELECT
dt.titulo,
dt.show_type,
dt.tmdb_score,
dt.tmdb_popularity,
dt.ano_lancamento
FROM tabela_pais tp
JOIN dim_pais dp
ON tp.id_pais = dp.id_pais
JOIN dim_titulo dt
ON tp.id_titulo = dt.id_titulo
WHERE dp.pais = 'BR'
AND dt.tmdb_score IS NOT NULL
ORDER BY
dt.tmdb_score DESC,
dt.tmdb_popularity DESC
LIMIT 10;


Os resultados indicam que os títulos brasileiros mais populares apresentam boa convergência entre IMDb e TMDb, com predominância de séries entre os destaques. No IMDb, sobressaem produções com notas elevadas (acima de 7) e volumes moderados de votos, refletindo avaliações positivas, porém com alcance mais restrito quando comparadas a produções internacionais.

No TMDb, observa-se maior variação na popularidade, mesmo entre títulos com pontuações semelhantes, evidenciando diferenças de visibilidade e engajamento do público. Alguns títulos mantêm bom desempenho em ambas as plataformas, enquanto outros se destacam mais pelo alcance (TMDb) do que pela avaliação média (IMDb). Em conjunto, os resultados mostram que a produção brasileira possui reconhecimento consistente, especialmente no formato de séries, ainda que com níveis de popularidade distintos entre as plataformas.

**Ranking dos países com maior quantidade de títulos, filmes e séries (Top 10)**

In [0]:
SELECT
dp.pais,
COUNT(DISTINCT tp.id_titulo) AS total_titulos
FROM tabela_pais tp
JOIN dim_pais dp
ON tp.id_pais = dp.id_pais
GROUP BY dp.pais
ORDER BY total_titulos DESC
LIMIT 10;


In [0]:
SELECT
dp.pais,
COUNT(DISTINCT dt.id_titulo) AS total_filmes
FROM tabela_pais tp
JOIN dim_pais dp
ON tp.id_pais = dp.id_pais
JOIN dim_titulo dt
ON tp.id_titulo = dt.id_titulo
WHERE dt.show_type = 'MOVIE'
GROUP BY dp.pais
ORDER BY total_filmes DESC
LIMIT 10;


In [0]:
SELECT
dp.pais,
COUNT(DISTINCT dt.id_titulo) AS total_series
FROM tabela_pais tp
JOIN dim_pais dp
ON tp.id_pais = dp.id_pais
JOIN dim_titulo dt
ON tp.id_titulo = dt.id_titulo
WHERE dt.show_type = 'SHOW'
GROUP BY dp.pais
ORDER BY total_series DESC
LIMIT 10;


Os resultados evidenciam uma forte concentração de títulos nos Estados Unidos, que lideram de forma expressiva o ranking geral, bem como os rankings específicos de filmes e séries, indicando ampla predominância na produção e presença no catálogo. Em segundo e terceiro lugares aparecem Reino Unido e França, com volumes significativamente menores, mas ainda relevantes no contexto internacional.

No recorte por filmes, o padrão se mantém, com os Estados Unidos concentrando a maior parte das produções, seguidos por países europeus tradicionais no mercado audiovisual. Já no ranking de séries, observa-se uma maior diversidade geográfica, com a presença de países como Brasil, Espanha, México, Taiwan e Singapura, ainda que com quantidades bem inferiores ao líder.

De forma geral, o ranking mostra que, enquanto a produção de filmes é altamente concentrada, a produção de séries apresenta maior dispersão entre países, refletindo a expansão recente de conteúdos seriados em diferentes mercados.

**Ranking dos títulos mais populares por gênero**


In [0]:
WITH ranking_genero_imdb AS (
    SELECT
        dg.genero,
        dt.titulo,
        dt.show_type,
        dt.imdb_score,
        dt.imdb_votes,
        dt.ano_lancamento,
        ROW_NUMBER() OVER (
            PARTITION BY dg.genero
            ORDER BY dt.imdb_score DESC, dt.imdb_votes DESC
        ) AS rn
    FROM tabela_genero tg
    JOIN dim_genero dg
        ON tg.id_genero = dg.id_genero
    JOIN dim_titulo dt
        ON tg.id_titulo = dt.id_titulo
    WHERE dt.imdb_score IS NOT NULL
)

SELECT
genero,
titulo,
show_type,
imdb_score,
imdb_votes,
ano_lancamento
FROM ranking_genero_imdb
WHERE rn = 1
ORDER BY genero;


In [0]:
WITH ranking_genero_tmdb AS (
    SELECT
        dg.genero,
        dt.titulo,
        dt.show_type,
        dt.tmdb_score,
        dt.tmdb_popularity,
        dt.ano_lancamento,
        ROW_NUMBER() OVER (
            PARTITION BY dg.genero
            ORDER BY dt.tmdb_score DESC, dt.tmdb_popularity DESC
        ) AS rn
    FROM tabela_genero tg
    JOIN dim_genero dg
        ON tg.id_genero = dg.id_genero
    JOIN dim_titulo dt
        ON tg.id_titulo = dt.id_titulo
    WHERE dt.tmdb_score IS NOT NULL
)

SELECT
genero,
titulo,
show_type,
tmdb_score,
tmdb_popularity,
ano_lancamento
FROM ranking_genero_tmdb
WHERE rn = 1
ORDER BY genero;


De modo geral, os rankings mostram forte predominância de séries como os títulos mais populares em diversos gêneros, tanto no IMDb quanto no TMDb, indicando que o formato seriado tende a concentrar maior engajamento e reconhecimento do público. No IMDb, destacam-se produções com pontuações muito elevadas e grande volume de votos, como Band of Brothers, Chernobyl, The Wire e Game of Thrones, que aparecem como referência em múltiplos gêneros, evidenciando seu caráter transversal.

No TMDb, observa-se uma maior concentração de pontuações máximas (score 10) entre os gêneros, o que torna a popularidade (tmdb_popularity) o principal fator de diferenciação entre os títulos. Alguns conteúdos com nota máxima apresentam popularidade relativamente baixa, enquanto outros combinam alta avaliação e alto alcance, indicando diferenças relevantes de visibilidade dentro de um mesmo gênero.

Em síntese, enquanto o IMDb privilegia títulos com avaliações consolidadas e alto volume de votos, o TMDb evidencia melhor o alcance e a exposição dos conteúdos, reforçando a importância de utilizar métricas complementares para identificar os títulos mais representativos em cada gênero.

**Gêneros mais comuns na plataforma**

In [0]:
SELECT
dg.genero,
COUNT(DISTINCT tg.id_titulo) AS total_titulos
FROM tabela_genero tg
JOIN dim_genero dg
ON tg.id_genero = dg.id_genero
GROUP BY dg.genero
ORDER BY total_titulos DESC;


 

Os dados indicam que drama e comédia são os gêneros mais comuns na plataforma, concentrando a maior parte dos títulos do catálogo. Em seguida, destacam-se documentário, thriller e ação, evidenciando uma forte presença de conteúdos narrativos e de entretenimento. Gêneros como crime, romance e ficção científica apresentam participação intermediária, enquanto western, reality e esporte aparecem com menor representatividade. De forma geral, o ranking revela um catálogo predominantemente voltado a dramas e comédias, com diversidade de gêneros complementares, porém em menor escala.

**Filmes com maior duração**

In [0]:
SELECT
ROUND(AVG(duracao), 2) AS duracao_media_filmes
FROM dim_titulo
WHERE show_type = 'MOVIE'
AND duracao IS NOT NULL;


In [0]:
SELECT
titulo,
duracao,
ano_lancamento,
imdb_score,
tmdb_score
FROM dim_titulo
WHERE show_type = 'MOVIE'
AND duracao IS NOT NULL
ORDER BY duracao DESC
LIMIT 10;


A duração média dos filmes disponíveis na plataforma é de aproximadamente 94 minutos, indicando um catálogo predominantemente composto por longas-metragens de duração padrão, próximos ao formato tradicional de cinema comercial.

Em contraste, o ranking dos filmes com maior duração evidencia a presença de produções excepcionais, com tempos de exibição muito acima da média, incluindo edições estendidas, documentários e épicos históricos, com durações superiores a 200 minutos. Esses títulos representam casos específicos e pouco frequentes no catálogo, não influenciando de forma significativa a média geral, mas demonstrando a diversidade de formatos e propostas narrativas disponíveis na plataforma.

**Séries com maior número de temporadas**

In [0]:
SELECT
CAST(ROUND(AVG(temporadas), 0) AS INT) AS temporadas_media_series
FROM dim_titulo
WHERE show_type = 'SHOW'
AND temporadas IS NOT NULL;



In [0]:
SELECT
titulo,
temporadas,
ano_lancamento,
imdb_score,
tmdb_score
FROM dim_titulo
WHERE show_type = 'SHOW'
AND temporadas IS NOT NULL
ORDER BY temporadas DESC
LIMIT 10;


A análise indica que as séries da plataforma possuem, em média, cerca de 3 temporadas, o que sugere um catálogo majoritariamente composto por produções de curta a média duração.

Em contraste, o ranking das séries com maior número de temporadas evidencia títulos longevos e consolidados, como Sesame Street, Top Gear e South Park, com mais de 20 temporadas, representando casos excepcionais dentro do conjunto total. Essas produções de longa duração coexistem com séries mais curtas, indicando diversidade no ciclo de vida dos conteúdos seriados, mas sem impactar significativamente a média geral.

**Países com maior diversidade de gêneros**

In [0]:
SELECT
dp.pais,
COUNT(DISTINCT dg.genero) AS total_generos
FROM tabela_pais tp
JOIN dim_pais dp
ON tp.id_pais = dp.id_pais
JOIN tabela_genero tg
ON tp.id_titulo = tg.id_titulo
JOIN dim_genero dg
ON tg.id_genero = dg.id_genero
GROUP BY dp.pais
ORDER BY total_generos DESC
LIMIT 10;


O ranking evidencia que Reino Unido (GB) e Estados Unidos (US) apresentam a maior diversidade de gêneros, cobrindo praticamente todo o espectro temático disponível na plataforma. Em seguida, países como Canadá, França, Japão, Alemanha e Espanha também demonstram alta variedade de gêneros, indicando catálogos equilibrados e diversificados. Já China, Austrália e México aparecem com diversidade ligeiramente menor, mas ainda relevante. De forma geral, os resultados mostram que países com maior maturidade e alcance na indústria audiovisual tendem a oferecer maior diversidade temática, refletindo capacidade de produção em múltiplos gêneros.

**Diretores com maior número de títulos disponíveis**

In [0]:
SELECT
dp.nome AS diretor,
COUNT(DISTINCT fc.id_titulo) AS total_titulos
FROM fato_creditos fc
JOIN dim_pessoa dp
ON fc.id_pessoa = dp.id_pessoa
WHERE fc.funcao = 'DIRECTOR'
GROUP BY dp.nome
ORDER BY total_titulos DESC
LIMIT 10;


O ranking evidencia os diretores com maior número de títulos disponíveis na plataforma, com destaque para Charlie Chaplin, que lidera com ampla margem, seguido por Sam Liu e Jon Alpert. Observa-se a presença de diretores clássicos e contemporâneos, indicando um catálogo que combina produções históricas e modernas. De forma geral, os resultados refletem a diversidade temporal e estilística do acervo, bem como a concentração de títulos em diretores com filmografias extensas.

**Atores que aparecem com mais frequência**

In [0]:
SELECT
dp.nome AS ator,
COUNT(DISTINCT fc.id_titulo) AS total_titulos
FROM fato_creditos fc
JOIN dim_pessoa dp
ON fc.id_pessoa = dp.id_pessoa
WHERE fc.funcao = 'ACTOR'
GROUP BY dp.nome
ORDER BY total_titulos DESC
LIMIT 10;


O ranking evidencia os atores com maior frequência de aparições no catálogo, com destaque para Grey DeLisle e Frank Welker, que lideram com ampla margem, refletindo carreiras extensas, sobretudo em produções de animação e dublagem. Observa-se ainda a presença recorrente de atores especializados em vozes e personagens seriados, o que explica o elevado número de participações. De forma geral, os resultados indicam que a frequência de aparições está fortemente associada à atuação em conteúdos seriados, mais do que em produções isoladas.

## **Conclusão**

A partir das análises realizadas sobre o catálogo de filmes e séries, foi possível responder de forma consistente a todas as perguntas definidas inicialmente no projeto, validando a qualidade do pipeline de dados e do modelo dimensional construído.

Os resultados demonstram que a popularidade dos títulos varia significativamente conforme a métrica utilizada. Enquanto o IMDb reflete avaliações diretas do público (score e volume de votos), o TMDb complementa a análise ao incorporar um indicador de popularidade mais dinâmico, permitindo desempates e uma visão mais atualizada de engajamento. Essa combinação mostrou-se essencial para evitar interpretações enviesadas.

Observou-se que:

- Filmes e séries mais populares tendem a ser produções consolidadas, com grande volume de votos e reconhecimento histórico.

- Títulos menos populares, em geral, apresentam baixa quantidade de avaliações, o que indica menor alcance e não necessariamente baixa qualidade.

- O Brasil possui presença relevante no catálogo, com produções bem avaliadas, especialmente no formato de séries, embora com menor volume quando comparado a países como Estados Unidos e Reino Unido.

- Os Estados Unidos dominam amplamente o volume total de títulos, tanto em filmes quanto em séries, além de apresentarem alta diversidade de gêneros.
 
- Drama, comédia e documentário são os gêneros mais recorrentes na plataforma, indicando preferência por conteúdos narrativos e informativos.

- A duração média dos filmes é de aproximadamente 94 minutos, enquanto séries apresentam, em média, cerca de 3 temporadas, evidenciando um padrão de produções relativamente compactas.

- Alguns títulos se destacam estruturalmente, como filmes de longa duração e séries com elevado número de temporadas, refletindo longevidade e sucesso contínuo.
 
- No eixo de créditos, foi possível identificar diretores e atores com maior presença no catálogo, reforçando padrões de recorrência e produtividade artística.

De forma geral, as análises confirmam que o modelo de dados construído foi adequado para responder perguntas estratégicas sobre popularidade, diversidade, produção e estrutura do catálogo, cumprindo plenamente os objetivos do MVP.